In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [3]:
# Read the .csv file
df_isuzu = pd.read_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\\raw_datasets\isuzuR.csv")

<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_18208\2313575260.py:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
  df_isuzu = pd.read_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\\raw_datasets\isuzuR.csv")


In [4]:
# Check the cols available
df_isuzu.columns

Index(['ModType', 'Hardware', 'Isuzu PN', 'GM PN', 'BCC', 'GM Serv №',
       'Sticker Description', 'VIN Example', 'Year', 'Make & Mdl', 'Engine',
       'Trans.', 'EPC Part Number', 'Hardware ID'],
      dtype='object')

In [5]:
# Print the head
df_isuzu.head(2)

,ModType,Hardware,Isuzu PN,GM PN,BCC,GM Serv №,Sticker Description,VIN Example,Year,Make & Mdl,Engine,Trans.,EPC Part Number,Hardware ID
0,IS-U38B,U38B-63A,8162152393,16215239,BUWP,16205463,3.2S AUT PCM,NaN,1996,Isuzu Rodeo,3.2L,NaN,NaN,892809268
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996,Honda Passport,3.2L,NaN,NaN,892809268


In [6]:
# Get the index with empty rows
df_isuzu_empty_rows_index = df_isuzu[df_isuzu['ModType'].isna() == True].index

In [7]:
# Drop the empty rows
df_isuzu_drop_empty = df_isuzu.drop(df_isuzu_empty_rows_index)

In [8]:
# Drop the repeated columns
df_isuzu_repeated_cols_index = df_isuzu_drop_empty[df_isuzu_drop_empty['ModType'] == 'ModType'].index

In [9]:
# Drop the repeated cols
df_isuzu_repeated_cols_dropped = df_isuzu_drop_empty.drop(df_isuzu_repeated_cols_index)

In [10]:
# Remove rows where all cols have null (NaN) values,
# keeping only rows with at least one valid (non-null) entry.
df_isuzu_rm_null_rows = df_isuzu_repeated_cols_dropped[~df_isuzu_repeated_cols_dropped.isnull().all(axis=1)]

In [11]:
# Keep only rows where more than one col has a non-null value.
# This removes rows with only a single non-null entry, ensuring each row has multiple valid data points.
df_isuzu_rm_one_null_row = df_isuzu_rm_null_rows[df_isuzu_rm_null_rows.notnull().sum(axis=1) != 1]

In [12]:
# Show cols
df_isuzu_rm_one_null_row.columns

Index(['ModType', 'Hardware', 'Isuzu PN', 'GM PN', 'BCC', 'GM Serv №',
       'Sticker Description', 'VIN Example', 'Year', 'Make & Mdl', 'Engine',
       'Trans.', 'EPC Part Number', 'Hardware ID'],
      dtype='object')

# Cols to keep:
- Hardware
- ModType
- Isuzu PN
- GM PN
- GM Serv №

In [13]:
# List with cols to keep
cols_to_keep = [
    "Hardware", 
    "ModType",
    "Isuzu PN",
    "GM PN",
    "GM Serv №"
]

In [14]:
# Create a df with needed cols
df_isuzu_needed_cols = df_isuzu_rm_one_null_row[cols_to_keep]

In [15]:
def fill_null_rows(df):
    '''
    Iterate over all columns and fill out all null values.

    Parameters: 
        df (DataFrame): main df.

    Returns:
        df: Return a df with the null values filled out
    '''
    for col in df.columns:
        df[col] = df[col].fillna("Not Available")

    return df

In [16]:
# Function to remove the whitespaces
def remove_whitespaces(df):
    # Loop to iterate over all cols
    for col in df.columns:
        # Remove the writespaces
        df[col] = df[col].str.strip()

    return df

In [17]:
# Function to split multiple part numbers originally in the same line to one row each
def split_part_numbers(df, col):

    # Call the function to remove the whitespaces
    df_rm_spaces = remove_whitespaces(df)

    df_copy = df_rm_spaces.copy()

    # Create a list with the items in each row 
    df_copy[col] = df_copy[col].str.split(",")
    # Explode the items in different rows each and keep the info from the other rows
    df_splitted = df_copy.explode(col)
    # Return the df with the exploded part numbers
    return df_splitted

In [18]:
# Call the function to fill out null values
df_isuzu_fill_nulls = fill_null_rows(df_isuzu_needed_cols)

C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_18208\185194438.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna("Not Available")


In [19]:
# Show the available cols
df_isuzu_needed_cols.columns

Index(['Hardware', 'ModType', 'Isuzu PN', 'GM PN', 'GM Serv №'], dtype='object')

### Call the function to the explode all items of the cols

In [20]:
df_isuzu_expl_hdw_col = split_part_numbers(df_isuzu_needed_cols, 'Hardware')

C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_18208\3366128590.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.strip()


In [21]:
df_isuzu_expl_modtype_col = split_part_numbers(df_isuzu_expl_hdw_col, 'ModType')

In [22]:
df_isuzu_expl_isuzuPn_col = split_part_numbers(df_isuzu_expl_modtype_col, 'Isuzu PN')

In [23]:
df_isuzu_expl_gmPn_col = split_part_numbers(df_isuzu_expl_isuzuPn_col, 'GM PN')

In [24]:
df_isuzu_expl_servN_col = split_part_numbers(df_isuzu_expl_gmPn_col, 'GM Serv №')

## Standardizing Part Number Dataset Columns
Each part number dataset contains columns with different names because they originate from different manufacturers. One of the goals of cleaning these datasets is to create a standard format that allows for the identification of the hardware part number when inputting other part numbers, such as software part numbers or system part numbers.

Since the identification of the hardware part number will be performed by inputting multiple part numbers at once (via a .csv file), it is necessary — except for the hardware column — to rename the other columns to a standard format: `pnNum` (e.g., `pn1`, `pn2`, etc.). This ensures consistency and enables automated matching and processing across datasets from various manufacturers.

In [25]:
def rename_col_names(df):
    '''
    Except the column hardware, the function standardize the names for pnNumber (pn1, pn2).
    Needed due to each part number dataset has different column names, so that 

    Parameters:
    df (DataFrame): main df.
    col_list (list): List of original column names.

    Returns
    df: Return the main df with the renamed cols

    '''

    # List with the col names
    col_name_list = list(df.columns) 
    
    # Dict to append the old names (key): new names (values) 
    new_cols_dict = {}

    # Count the items under the col list
    col_num = len(col_name_list)
    # Loop to iterate over the list
    for col in col_name_list:
        # Condition to define when lower the col name and when rename it
        if col == "hardware":
            new_cols_dict[col] = col.lower()
        else:
            # Convert the string to number to sum with the stirng pn and
            # append as value to the dict
            new_cols_dict[col] = "pn" + str(col_num)
            # Return the number from string to int
            col_num = int(col_num)
        
        # Subtract one to create the next col name
        num_col = col_num - 1
        col_num = num_col

    # Return the main df with the cols remanes
    return df.rename(columns=new_cols_dict)

In [26]:
# Map to rename 'Hardware' col
map_hdw_col = {
    "Hardware": "hardware"
}

In [27]:
# Rename 'Hardware' col
df_isuzu_rn_hdw_col = df_isuzu_expl_servN_col.rename(columns=(map_hdw_col))

In [28]:
# Rename all cols
df_isuzu_rn_cols = rename_col_names(df_isuzu_rn_hdw_col)

In [29]:
# Check df info
df_isuzu_rn_cols.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, 0 to 148
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hardware  101 non-null    object
 1   pn4       101 non-null    object
 2   pn3       101 non-null    object
 3   pn2       101 non-null    object
 4   pn1       101 non-null    object
dtypes: object(5)
memory usage: 4.7+ KB


In [88]:
# Export to .csv file
df_isuzu_rn_cols.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\data_cleaned\pn-isuzu.csv", index=False)

<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
<>:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_16128\2993268055.py:2: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
  df_isuzu_rn_cols.to_csv("C:\Language_Projects\Language_Projects\Python\Flagship_1\part-number-datasets-cleaning\data\data_cleaned\pn-isuzu.csv", index=False)


## Database Import Documentation

After cleaning the data and preparing the DataFrame as needed, the data will be imported into a database. The functions below are intended to connect to the database and import the datasets.

In [30]:
def import_datasets_to_db(df, df_name=str):

    '''
    Import the df cleaned to the db

    Parameters: 
        df (DataFrame): main df to import to the db.
        df_name: string to label the df into the db.

    Returns:
        Import the df to the db. 
    '''
    # Setting up the connection with the PosgreSQL
    dbname="prescreen_diag_data_api"
    user="postgres"
    password="shakey-10"
    host="localhost"
    port="5432"

    # String connection for SQLAlchemy (using psycopg2 as driver)
    engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

    return df.to_sql(df_name, engine, if_exists='replace', index=False)


In [31]:
# Call the function to import the df cleaned to the postgreSQLdb
import_datasets_to_db(df_isuzu_rn_cols, 'part_numbers_isuzu')

101